In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# Utilities 
from options.options import is_notebook, get_options, update_options, print_options
import argparse
import math
import time
import random
import os
from tqdm import tqdm

# Data
import numpy as np
from sklearn.model_selection import train_test_split
from datautils.ts_dataset import TSDataset
from datautils.data import prepare_dataloaders
import pandas as pd

# Torch
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Transformer
import transformer.Constants as Constants
from transformer.Models import Transformer
from transformer.Optim import ScheduledOptim

%autoreload 2

In [ ]:
opt = get_options()
print_options(opt)

In [ ]:
# https://pytorch.org/docs/stable/notes/randomness.html
# For reproducibility
if opt["seed"] is not None:
    torch.manual_seed(opt["seed"])
    torch.backends.cudnn.benchmark = False
    # torch.set_deterministic(True)
    np.random.seed(opt["seed"])
    random.seed(opt["seed"])

In [ ]:
if not opt["output_dir"]:
    print('No experiment result will be saved.')
    raise

In [ ]:
if not os.path.exists(opt["output_dir"]):
    os.makedirs(opt["output_dir"])

In [ ]:
# define the cuda devide
device = torch.device('cuda' if opt["cuda"] else 'cpu')

In [ ]:
# Load the dataset and preprocess it 
aq = pd.read_parquet("/home/yannic/master-thesis/data_air/prsa_data.parquet")
aq_prep = pd.concat([aq,pd.get_dummies(aq['station'], prefix='station',dummy_na=False)],axis=1).drop(['station'],axis=1).drop(columns=["wind_direction"])

In [ ]:
#========= Loading Dataset =========#

train, test = train_test_split(aq_prep, test_size=0.10)

train_dataset, train_dataloader = prepare_dataloaders(train.values, opt["batch_size"])
test_dataset, test_dataloader = prepare_dataloaders(test.values,  opt["batch_size"])



# Define Transformer
transformer = Transformer(
        opt.src_vocab_size,
        opt.trg_vocab_size,
        src_pad_idx=opt.src_pad_idx,
        trg_pad_idx=opt.trg_pad_idx,
        trg_emb_prj_weight_sharing=opt.proj_share_weight,
        emb_src_trg_weight_sharing=opt.embs_share_weight,
        d_k=opt.d_k,
        d_v=opt.d_v,
        d_model=opt.d_model,
        d_word_vec=opt.d_word_vec,
        d_inner=opt.d_inner_hid,
        n_layers=opt.n_layers,
        n_head=opt.n_head,
        dropout=opt.dropout,
        scale_emb_or_prj=opt.scale_emb_or_prj).to(device)

# # Define Optimizer
# optimizer = ScheduledOptim(
#                 optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
#                 opt.lr_mul, opt.d_model, opt.n_warmup_steps)
# 
# # Start the training process
# train(transformer, training_data, validation_data, optimizer, device, opt)

# Functions

In [85]:
from tstransformer.Layers import EncoderLayer, DecoderLayer
from tstransformer.Models import PositionalEncoding
%autoreload 2

In [92]:
pos = PositionalEncoding(2,4)

In [94]:
pos.pos_table

tensor([[[ 0.0000,  1.0000],
         [ 0.8415,  0.5403],
         [ 0.9093, -0.4161],
         [ 0.1411, -0.9900]]])

In [ ]:
size